In [1]:
import sys
if not '..' in sys.path:
    sys.path.append('..')
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colormaps
from analysis import collection, analysis, emg
import scipy as sp
import pandas as pd
import swifter

from tqdm import tqdm
tqdm.pandas()

from swifter import set_defaults
set_defaults(
    npartitions=12,
    dask_threshold=2,
    scheduler="processes",
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=False,
    force_parallel=True,
)

/home/spencer/miniconda3/envs/data/lib/python3.12/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/home/spencer/miniconda3/envs/data/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
channels = np.arange(2)
n_timepoints = 2000
timepoints = np.arange(n_timepoints)
n_targets = 12
target_numbers = range(1,n_targets+1)
n_blocks = 45
blocks = range(n_blocks)
n_subjects = 46
subjects = range(n_subjects)

multi_index = pd.MultiIndex.from_product([subjects,blocks,target_numbers,timepoints],names=["subject","block","target","time"])
print(multi_index.shape)
subject_traj_stacks = np.load("subject_filtered_trajectory_stacks.npy").transpose(0,1,2,4,3)
print(subject_traj_stacks.shape)
df = pd.DataFrame(subject_traj_stacks.reshape(-1,2),index=multi_index,columns=channels).dropna()
del subject_traj_stacks
print(df.shape)

(49680000,)
(46, 45, 12, 2000, 2)
(7989403, 2)


In [3]:
# make everything numpy for HUGE speedups!
target_vecs = np.array([collection.compute_target_vec_from_number(target_number) for target_number in range(1,13)])

def compute_reward(row):
    return np.linalg.norm(row.to_numpy() - target_vecs[row.name[2] - 1])

In [4]:
df["reward"] = df.swifter.apply(compute_reward,axis='columns',result_type="expand")

Pandas Apply: 100%|██████████| 7989403/7989403 [01:07<00:00, 118326.73it/s]
/tmp/ipykernel_14632/3822169844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reward"] = df.swifter.apply(compute_reward,axis='columns',result_type="expand")


In [16]:
subject_means = df.groupby(level=["subject"]).mean()["reward"].to_numpy()

In [17]:
subject_means

array([0.73099753, 0.91790218, 1.07413579, 0.83975535, 1.64866241,
       1.10260095, 0.64802526, 1.17657013, 1.1124183 , 1.06453238,
       0.98989446, 0.73068902, 1.31006926, 1.14054334, 0.87087456,
       0.83560084, 0.74009076, 0.81439326, 0.91261121, 1.10256106,
       1.31260078, 0.8731727 , 0.76335023, 2.03272531, 0.83820096,
       1.20565439, 1.47434129, 0.94514226, 1.10814163, 0.51741588,
       0.91681046, 0.95579013, 0.82276626, 0.82240779, 1.49835347,
       0.7867887 , 0.78904761, 0.70650151, 1.13983938, 1.31562858,
       0.9062382 , 0.86533955, 0.92182935, 1.01408642, 1.00768263,
       1.20706692])

In [19]:
np.argsort(subject_means**-1)[::-1]

array([29,  6, 37, 11,  0, 16, 22, 35, 36, 17, 33, 32, 15, 24,  3, 41, 14,
       21, 40, 18, 30,  1, 42, 27, 31, 10, 44, 43,  9,  2, 19,  5, 28,  8,
       38, 13,  7, 25, 45, 12, 20, 39, 26, 34,  4, 23])

In [20]:
df.to_pickle("trajectory_dataframe.pkl")

In [21]:
np.save("subject_rewards.npy",subject_means**-1)